# Text index

In [ ]:
import asyncio
import json
import math
import os
import pickle
import re
import unicodedata

import faiss
import nest_asyncio
import numpy as np
import pandas as pd
import torch
from PIL import Image
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from stopwordsiso import stopwords
from telethon import TelegramClient
from transformers import (
    CLIPProcessor,
    CLIPModel,
    BlipForConditionalGeneration,
    BlipProcessor,
)

In [38]:
# Показывать все столбцы
pd.set_option('display.max_columns', None)
# Не обрезать ширину таблицы по общему числу символов
pd.set_option('display.width', None)
# Показывать весь текст в ячейках (не обрезать по ширине)
pd.set_option('display.max_colwidth', None)


# 1. Загрузка
with open("news_data2.json", encoding="utf-8") as f:
    raw = json.load(f)
df = pd.DataFrame(raw)
df.head(1)


,category,channel,message_id,time,text,image_path
0,General news,@rian_ru,291023,25.04.2025,Разведка Индии на фоне обострения отношений с Пакистаном предупредила о возможности новых терактов в Джамму и Кашмире,


In [ ]:
old_text = ""
new_text = " "

# Заменяем None/NaN на пустую строку
df["text"] = df["text"].fillna("")

# Применяем replace только к тем строкам, у которых длина < 2
df["text"] = [
    path.replace(old_text, new_text) if len(path) < 2 else path
    for path in df["text"]
]

# 3. Предобработка
ru_stop = stopwords("ru")
def clean(text: str) -> str:
    text = BeautifulSoup(text, "html.parser").get_text(" ", strip=True)
    text = unicodedata.normalize("NFKC", text).lower()
    text = re.sub(r"[^\w\s]", " ", text)
    tokens = [t for t in text.split() if t not in ru_stop]
    return " ".join(tokens)
df["clean_text"] = df["text"].map(clean)

# 4. Фильтрация коротких новостей
df["word_count"] = df["clean_text"].str.split().apply(len)
df = df[df["word_count"] > 7].reset_index(drop=True)
df = df.drop(columns=["word_count"])

months = {
    '01': 'января', '02': 'февраля', '03': 'марта', '04': 'апреля',
    '05': 'мая', '06': 'июня', '07': 'июля', '08': 'августа',
    '09': 'сентября', '10': 'октября', '11': 'ноября', '12': 'декабря'
}
df['time_'] = df['time'].str.replace(
    r'(\d{2})\.(\d{2})\.\d{4}',
    lambda m: f"{int(m.group(1))} {months[m.group(2)]}",
    regex=True
)

df["clean_text"] = df["time_"] + " " + df["clean_text"]

df.head(1)

,category,channel,message_id,time,text,image_path,clean_text,time_
0,General news,@rian_ru,291023,25.04.2025,Разведка Индии на фоне обострения отношений с Пакистаном предупредила о возможности новых терактов в Джамму и Кашмире,,25 апреля разведка индии фоне обострения отношений пакистаном предупредила возможности новых терактов джамму кашмире,25 апреля


In [23]:
news_image_text = df[
    df["image_path"].notna()            # не NaN/None
    & df["image_path"].ne("")           # не пустая строка
]
news_image_text.head(1)

,category,channel,message_id,time,text,image_path,clean_text,time_
10,General news,@rian_ru,291006,25.04.2025,"На фоне обострения между Индией и Пакистаном РИА Новости сравнило военные потенциалы двух стран.\n\nОборонный бюджет Индии почти в 10 раз превышает расходы Пакистана на армию. В авиации, живой силе и танках у Индии преимущество примерно в два раза.\n\nПри этом по числу ядерных боезарядов у Исламабада и Нью-Дели практически паритет.",/content/drive/MyDrive/Diploma/images/rian_ru_291006.jpg,25 апреля фоне обострения индией пакистаном риа новости сравнило военные потенциалы стран оборонный бюджет индии 10 превышает расходы пакистана армию авиации живой силе танках индии преимущество примерно раза числу ядерных боезарядов исламабада нью дели практически паритет,25 апреля


In [24]:
news_image_paths = df[
    df["image_path"].notna()            # не NaN/None
    & df["image_path"].ne("")           # не пустая строка
]
news_image_paths = list(news_image_paths["image_path"])

In [25]:
len(news_image_paths[0])

56

In [26]:
# Преобразуем Series в обычный список
texts = df["clean_text"].tolist()

# Сформируем нужную структуру
data = {
    "clean_text": texts
}

# Сохраним в файл в человекочитаемом виде с Unicode напрямую
with open("gpt_news.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [27]:
from sentence_transformers import SentenceTransformer

text = df['clean_text']
encoder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2") # bert-base-nli-mean-tokens


In [28]:
embeddings = encoder.encode(text)

In [29]:
embeddings

array([[-0.09420574, -0.00633919, -0.13923542, ...,  0.17324284,
        -0.04576061, -0.01844954],
       [-0.0217472 ,  0.25407398, -0.15008026, ...,  0.21548347,
        -0.3479332 ,  0.09033651],
       [-0.14909697,  0.00256209, -0.04424019, ..., -0.04633314,
        -0.14459951, -0.09079548],
       ...,
       [-0.31644922,  0.24303204,  0.04689507, ..., -0.06389456,
         0.11051159,  0.22291371],
       [-0.21578182,  0.10785401,  0.01316834, ..., -0.01632707,
         0.01277807, -0.02064398],
       [-0.20739281,  0.078646  ,  0.16746822, ..., -0.035949  ,
         0.11581522,  0.04504444]], shape=(5199, 384), dtype=float32)

In [30]:
embeddings.shape

(5199, 384)

In [31]:
import faiss
vector_dimensions = embeddings.shape[1]

index = faiss.IndexFlatL2(vector_dimensions)
faiss.normalize_L2(embeddings)
index.add(embeddings)

In [32]:
faiss.write_index(index, "news_date.faiss")

In [33]:
import numpy as np

search_text = '16 апреля Новости про Украину '
search_vector = encoder.encode(search_text)
new_vector = np.array([search_vector])
faiss.normalize_L2(new_vector)
distances, ann = index.search(new_vector, k=5)
indices = ann[0]
top_texts = df['clean_text'].iloc[indices].reset_index(drop=True)
results = pd.DataFrame({
    'distances': distances[0],
    'clean_text': top_texts
})

results

,distances,clean_text
0,0.439951,13 сентября одна причин украина пользуется поддержкой странах rt госдеп зря прожила серьезно
1,0.461347,23 апреля заявления пескова урегулирования украине нюансов сближать позиции продолжается сша продолжают посреднические усилия урегулированию украине приветствует расценивает слова вэнса возможном выходе сша переговорного процесса украине ультиматум стороны вашингтона любые наброски вариантов украинского урегулирования публичными иначе потеряют эффективность прежнему выступает европейских миротворцев украине определилась представлять похоронах папы римского генсек оон планирует приезжать москву 80 летие победы путин среду проведет заседание военно промышленной комиссии
2,0.489277,22 июля вашингтон брюссель осуществляют геноцид украинского народа согласно идеологии сотни жителей украины должны погибнуть фронте киевскому режиму поставляется большее количество вооружения остальные потерять национальную идентичность решения задачи используются имперские амбиции польши зеленский принял решений ставших огромной трагедией украинского народа закон фактически приравняет поляков украинцам предоставив права нахождение территории страны разрешений трудоустройство обучение медобслуживание некоторые выплаты приграничных россией областях сумской черниговской родных домов выгоняют украинцев стремясь заменить лояльных нацистам переселенцев поляков гражданам украины вместо русского навязывается польский язык включён список экзаменов вузы организуются гонения православную веру украинцев обращают католичество одним словом зеленский ведёт процесс ополячивания страны надеясь благодарность хозяева дадут возможность подольше удержаться власти фоне создано 25 тысячное воинское соединение состоящее основном польских литовских вооружённых солдат готовых оккупации западной украины поляки хотят вернуть считают исторические земли восточные кресы граждане украины территориях нужны вашингтон брюссель продолжат вести войну последнего украинца
3,0.499606,20 октября территорий является первостепенным украины урегулировании конфликта россией the telegraph восприняла слова украинского посла великобритании бывшего главкома всу валерия залужного сильно меняется риторика интересно подписаться прислать новость
4,0.502487,16 мая киевский режим втягивает сша страны европы большую войну депутаты рады украины пытаются убедить вашингтон разрешить применение поставляемых сша вооружений поражения целей территории россии ведёт трагедии затронуть человечество киевский режим проигрывая поле боя пытается втянуть соединённые штаты страны европы большую войну любое использование американского европейского оружия нанесения ударов мирным городам российской федерации потребует применения мощного оружия защитить граждан нашей страны западным политикам необходимо осознать ответственность доводить ситуацию катастрофы мирового масштаба


In [ ]:
results

,distances,clean_text
0,0.440108,16 апреля украина привлекала боевым действиям угледаре днр наемников арабских стран сообщил риа новости житель города
1,0.489277,22 июля вашингтон брюссель осуществляют геноцид украинского народа согласно идеологии сотни жителей украины должны погибнуть фронте киевскому режиму поставляется большее количество вооружения остальные потерять национальную идентичность решения задачи используются имперские амбиции польши зеленский принял решений ставших огромной трагедией украинского народа закон фактически приравняет поляков украинцам предоставив права нахождение территории страны разрешений трудоустройство обучение медобслуживание некоторые выплаты приграничных россией областях сумской черниговской родных домов выгоняют украинцев стремясь заменить лояльных нацистам переселенцев поляков гражданам украины вместо русского навязывается польский язык включён список экзаменов вузы организуются гонения православную веру украинцев обращают католичество одним словом зеленский ведёт процесс ополячивания страны надеясь благодарность хозяева дадут возможность подольше удержаться власти фоне создано 25 тысячное воинское соединение состоящее основном польских литовских вооружённых солдат готовых оккупации западной украины поляки хотят вернуть считают исторические земли восточные кресы граждане украины территориях нужны вашингтон брюссель продолжат вести войну последнего украинца
2,0.494198,15 апреля заявления пескова украинскому урегулированию идет достаточно напряженная положительные результаты сиюминутных стоит четких очертаний соглашения украинскому урегулированию политическая воля двигаться сторону экономическое сотрудничество россии сша сыграть стабилизирующую роль очевидно рада москве готов разделить гордость радость дня победы ожидает парад победы 20 глав правительств архиважный разговора трампом течение ближайших нескольких дней графике путина слова главы евродипломатии каллас высказалась присутствии представителей ес москве расценивать весьма весьма жесткие
3,0.499606,20 октября территорий является первостепенным украины урегулировании конфликта россией the telegraph восприняла слова украинского посла великобритании бывшего главкома всу валерия залужного сильно меняется риторика интересно подписаться прислать новость
4,0.502487,16 мая киевский режим втягивает сша страны европы большую войну депутаты рады украины пытаются убедить вашингтон разрешить применение поставляемых сша вооружений поражения целей территории россии ведёт трагедии затронуть человечество киевский режим проигрывая поле боя пытается втянуть соединённые штаты страны европы большую войну любое использование американского европейского оружия нанесения ударов мирным городам российской федерации потребует применения мощного оружия защитить граждан нашей страны западным политикам необходимо осознать ответственность доводить ситуацию катастрофы мирового масштаба


In [ ]:
def retrieve_similar_documents(search_text: str, k: int = 5):
    """Возвращает список из k наиболее релевантных текстовых фрагментов для заданного запроса."""
    search_vector = encoder.encode(search_text)
    new_vector = np.array([search_vector])
    faiss.normalize_L2(new_vector)
    distances, ann = index.search(new_vector, k=5)
    indices = ann[0]
    top_texts = df['clean_text'].iloc[indices].reset_index(drop=True)
    results = pd.DataFrame({
        'distances': distances[0],
        'clean_text': top_texts
    })
    return results

# Пример использования:
user_query = "Сколько Академику Алексею Матвеевичу Липанову?"
results = retrieve_similar_documents(user_query, k=10)
results

,distances,clean_text
0,0.755482,20 марта 20 марта 1980 президиум ан ссср принял постановление no 314 организации куйбышеве филиала физического института имени п н лебедева ан ссср самарский куйбышевский филиал фиан организован 1980 совместной инициативе областного руководства лауреата нобелевской премии академика николая геннадиевича басова решения фундаментальных прикладных задач области создания новых лазерных систем технологий первым директором филиала дважды лауреат государственной премии доктор физ мат наук профессор виктор анатольевич катулин руководством молодой коллектив провёл огромную работу короткий срок филиал обеспечен установками производственными площадями получены новые научные результаты лазерной технологии технологическим лазерам решены актуальные фундаментальные инженерные задачи установлено плодотворное сотрудничество вузами предприятиями нии нашей стране рубежом учёный вспоминал филиал начинается высадки волжские берега московского научного десанта группы молодых физиков воспитанников фиана кандидатов наук творческих дерзких исследователей ядро костяк коллектива предоставлено двухэтажное здание места теоретиков группы мощными лазерами предоставляет университет ближайшее предполагается строительство комплекса зданий института берегу волги лазерный комплекс расположится неподалёку университета интервью виктора анатольевича катулина волжская заря номер 24 07 1980 сф фиан живёт прочитать выше
1,0.797171,26 марта нападающий авангарда илья каблуков первым хоккеистом истории кхл принявшим участие 16 кубках гагарина 15 участий василия кошечкина евгения бирюкова кирилла петрова егора аверина ранее 37 летний форвард третьим хоккеистом проведшим 1000 матчей лиге лидером играм является вадим шипачёв 1036 1000 преодолел евгений бирюков 1001 завершивший карьеру pro хоккей
2,0.814344,15 апреля овечкин заработал истории хоккея статистический портал sportrac приводит следующие цифры ови получил карьеру нхл 161 72 благодаря контракта вашингтоном доходы россиянина вырастут 170 72 тройку заработку вошли сидни кросби 155 89 млн евгений малкин 146 87 млн подписаться sportrian
3,0.839260,11 апреля 1 2 3 хоккейный агент шуми бабаев представляющий интересы нападающего ска сергея толчинского отреагировал слухи возможном уходе форварда армейского клуба информация соответствуют действительности сергея действующий контракт роман борисович чётко дал рассчитывает игрока следующем сезоне лидера толчинского сильного выступления ухода ска обсуждался обсуждается напомним контракт толчинского рассчитан 31 мая 2028 ранее форвард подписал пятилетний контракт прошло сезоне 2024 2025 толчинский провёл 51 матч набрал 16 7 9 очков pro хоккей
4,0.880265,17 апреля поздравляем владимира михайловича алпатова 80 летием труды стали неотъемлемой частью лингвистического ландшафта россии научное наследие академика вдохновляет исследователей способствует развитию науки желаем здоровья благополучия юбилеи_ран


# CLIP index

In [8]:
import torch
from transformers import CLIPProcessor, CLIPModel,  CLIPTokenizer
from transformers import BlipProcessor, BlipForConditionalGeneration


# Загрузка CLIP модели для работы с изображениями
print("Загрузка CLIP-модели...")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

clip_index = faiss.read_index("clip_index_last.faiss" )

print("CLIP-модель загружена.")

Загрузка CLIP-модели...
CLIP-модель загружена.


In [20]:
with open("clip_valid_paths.pkl", "rb") as f:
    clip_valid_paths = pickle.load(f)
clip_valid_paths

['/content/drive/MyDrive/Diploma/images/rian_ru_290404.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290403.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290398.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290387.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290379.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290371.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290359.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290358.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290357.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290356.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290353.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290347.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290346.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290344.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290338.jpg',
 '/content/drive/MyDrive/Diploma/images/rian_ru_290335.jpg',
 '/content/drive/MyDrive

In [ ]:
# Старый путь, который нужно заменить
old_prefix = "/content/drive/MyDrive/Diploma/images/"
# Новый путь
new_prefix = "/Applications/Study/Diploma/images/"

# Обновляем все пути
clip_valid_paths = [path.replace(old_prefix, new_prefix) for path in clip_valid_paths]

In [ ]:
with open("clip_valid_paths.pkl", "wb") as f:
    pickle.dump(clip_valid_paths, f)

In [38]:
with open('news_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Предполагается, что JSON содержит ключи "text", "image_path" и "category"
news_texts = data["text"]
df = pd.DataFrame(data) 
news_image_paths = df[
    df["image_path"].notna()            # не NaN/None
    & df["image_path"].ne("")           # не пустая строка
]
news_image_paths = list(news_image_paths["image_path"])[0:5]


In [39]:
news_image_paths

['/Applications/Study/Diploma/images/rian_ru_289957.jpg',
 '/Applications/Study/Diploma/images/rian_ru_289954.jpg',
 '/Applications/Study/Diploma/images/rian_ru_289937.jpg',
 '/Applications/Study/Diploma/images/rian_ru_289934.jpg',
 '/Applications/Study/Diploma/images/rian_ru_289933.jpg']

In [ ]:
# ------------------------
# 3. Построение CLIP индекса (clip_index)
# ------------------------
clip_embeddings = []
clip_valid_paths = []  # Список успешно загруженных изображений

print("Вычисление эмбеддингов для изображений (CLIP)...")
for img_path in news_image_paths:
    try:
        image = Image.open(img_path).convert("RGB")
    except Exception as e:
        print(f"Ошибка при загрузке изображения {img_path}: {e}")
        continue
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        embedding = clip_model.get_image_features(**inputs)
    embedding = embedding.cpu().numpy().astype("float32")
    clip_embeddings.append(embedding[0])
    clip_valid_paths.append(img_path)

if clip_embeddings:
    clip_embeddings = np.vstack(clip_embeddings)
    d_clip = clip_embeddings.shape[1]
    clip_index = faiss.IndexFlatL2(d_clip)
    clip_index.add(clip_embeddings)
    faiss.write_index(clip_index, "clip_index_last.faiss")
    print(f"Число изображений в CLIP индексе: {clip_index.ntotal}")
else:
    print("Нет доступных изображений для формирования CLIP индекса.")

Вычисление эмбеддингов для изображений (CLIP)...
Число изображений в CLIP индексе: 5


In [58]:
import faiss, numpy as np
TEXT_INDEX_PATH = "news_date.faiss"    # FAISS index for text embeddings
CLIP_INDEX_PATH = "clip_index_last.faiss"         # FAISS index for image (CLIP) embeddings
BLIP_INDEX_PATH = "blip_index_last.faiss"         # FAISS index for BLIP caption embeddings
CLIP_PATHS_PATH = "clip_valid_paths.pkl"     # to store list of image paths corresponding to CLIP embeddings
BLIP_CAPTIONS_PATH = "blip_captions.pkl" 

# 3. Построение CLIP индекса (clip_index)
  # non-empty path
clip_embeddings_list = []
valid_paths = []
for img_path in news_image_paths:
    try:
        # Load image and compute CLIP image embedding
        image = Image.open(img_path).convert("RGB")
        inputs = clip_processor(images=image, return_tensors="pt")
        with torch.no_grad():
            img_emb = clip_model.get_image_features(**inputs)
        # Move embedding to CPU and convert to numpy array
        emb_np = img_emb.cpu().numpy().astype("float32")
        clip_embeddings_list.append(emb_np[0])
        valid_paths.append(img_path)
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        continue
if clip_embeddings_list:
    clip_embeddings = np.vstack(clip_embeddings_list)
    clip_index = faiss.read_index(CLIP_INDEX_PATH)
    clip_index.add(clip_embeddings)
    faiss.write_index(clip_index, CLIP_INDEX_PATH)
    # Save the list of valid image paths corresponding to the index vectors
    with open(CLIP_PATHS_PATH, "wb") as f:
        import pickle
        pickle.dump(valid_paths, f)
    print(f"Image index updated. Total images indexed: {clip_index.ntotal}")
else:
    # If no images or all failed
    print("No images available to update CLIP index.")



Image index updated. Total images indexed: 4330


In [57]:
clip_embeddings = np.vstack(clip_embeddings)

clip_index   = faiss.read_index("clip_index_last.faiss")

print("index.d        :", clip_index.d)          # чему обучался индекс
print("new vec shape :", clip_embeddings.shape)  # (N, d_new)
print("dtype          :", clip_embeddings.dtype) # должен быть float32

index.d        : 768
new vec shape : (5, 768)
dtype          : float32


In [ ]:
import pickle

# Сохраняем индекс Faiss
faiss.write_index(clip_index, "/content/drive/MyDrive/Diploma/clip_index_last.faiss")

# Сохраняем пути к изображениям, соответствующие эмбеддингам
with open("/content/drive/MyDrive/Diploma/clip_valid_paths.pkl", "wb") as f:
    pickle.dump(clip_valid_paths, f)

print("CLIP индекс и пути к изображениям успешно сохранены.")

In [45]:
i = faiss.read_index("clip_index_last.faiss")
i.ntotal

4320

# BLIP index

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Загрузка BLIP модели для генерации подписей к изображениям
print("Загрузка BLIP-модели...")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
print("BLIP-модель загружена.")

Загрузка BLIP-модели...
BLIP-модель загружена.


In [14]:
with open("blip_captions.pkl", "rb") as f:
    blip_captions = pickle.load(f)
blip_captions

['a map of the russian region of moscow with the most major cities',
 'a map of ukraine showing the location of the major cities',
 'a close up of a passport with a picture of a man',
 "a blue background with a white text that reads, ' russian hoboc '",
 'there is a man in a blue shirt and glasses sitting in a chair',
 "arafed image of a man in a priest ' s robes on a time magazine cover",
 'purple table with a purple cloth and a picture of a man with a candle',
 'woman sitting at a table writing in a book with a picture of a man',
 'a close up of a book with a picture of a pope on it',
 'a close up of a tree with the sun shining through the leaves',
 'flags of european union and european union flag flying in front of the european parliament',
 'arafed image of two men standing next to each other in a room',
 'purple flowers blooming on a tree in a city park',
 'arafed image of a priest standing in front of a microphone',
 'a close up of a document with a picture of a person on it',
 '

In [ ]:

# ------------------------
# 4. Построение BLIP индекса (blip_index)
# ------------------------
blip_captions = []
blip_valid_paths = []  # Аналогично, список корректных изображений

print("Генерация подписей для изображений (BLIP)...")
for img_path in news_image_paths:
    try:
        image = Image.open(img_path).convert("RGB")
    except Exception as e:
        print(f"Ошибка при загрузке изображения {img_path}: {e}")
        continue
    inputs = blip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        generated_ids = blip_model.generate(**inputs)
    caption = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    blip_captions.append(caption)
    blip_valid_paths.append(img_path)

print(f"Сгенерировано подписей для {len(blip_captions)} изображений.")

# Опционал ьно: создание FAISS индекса для эмбеддингов подписей с использованием модели SentenceTransformer
blip_caption_embeddings = model.encode(blip_captions, show_progress_bar=True)
blip_caption_embeddings = np.array(blip_caption_embeddings).astype("float32")
d_blip = blip_caption_embeddings.shape[1]
blip_index = faiss.IndexFlatL2(d_blip)
blip_index.add(blip_caption_embeddings)
print(f"Число подписей в BLIP индексе: {blip_index.ntotal}")


In [ ]:
import pickle

# Сохраняем индекс Faiss
faiss.write_index(blip_index, "/content/drive/MyDrive/Diploma/blip_index_last.faiss")

# Сохраняем пути к изображениям, соответствующие эмбеддингам
with open("/content/drive/MyDrive/Diploma/blip_captions.pkl", "wb") as f:
    pickle.dump(blip_captions, f)

print("CLIP индекс и пути к изображениям успешно сохранены.")

# Using CLIP+BLIP

In [16]:

# ------------------------
# 5. Функция поиска по изображениям (с использованием CLIP и BLIP)
# ------------------------

def truncate_query(query: str, max_length: int = 76) -> str:
    """
    Обрезает текстовый запрос до допустимой длины для CLIP-модели (максимум 77 токенов).
    """
    tokens = clip_tokenizer.tokenize(query)
    if len(tokens) > max_length - 2:  # учитываем специальные токены [CLS] и [SEP]
        tokens = tokens[:max_length - 2]
    return clip_tokenizer.convert_tokens_to_string(tokens)


def retrieve_similar_images(query: str, k: int = 5):
    """
    По заданному текстовому запросу с помощью CLIP модели
    ищет наиболее релевантные изображения из clip_index,
    после чего для найденных изображений возвращает соответствующие подписи (BLIP).
    Возвращается список кортежей (путь_к_изображению, подпись).
    """
    # Обрезаем запрос до допустимой длины
    safe_query = truncate_query(query)

    # Получаем текстовое представление запроса
    text_input = clip_processor(text=safe_query, return_tensors="pt")
    with torch.no_grad():
        text_features = clip_model.get_text_features(**text_input)
    text_features = text_features.cpu().numpy().astype("float32")

    # Поиск в CLIP индексе
    distances, indices = clip_index.search(text_features, k)
    retrieved = []
    for idx, dis in zip(indices[0], distances[0]):
        image_path = clip_valid_paths[idx]
        caption = blip_captions[idx]
        retrieved.append((image_path, caption, dis))
    return retrieved


sample_query = "Президент"
similar_images = retrieve_similar_images(sample_query, k=3)
print("Найденные изображения и подписи:")
for img_path, caption, distance in similar_images:
    print(f"Дистанция: {distance} - Изображение: {img_path} - Подпись: {caption}")


Найденные изображения и подписи:
Дистанция: 436.0837097167969 - Изображение: /Applications/Study/Diploma/images/igmtv_6807.jpg - Подпись: there is a piece of bread and a piece of cheese on a plate
Дистанция: 439.29296875 - Изображение: /Applications/Study/Diploma/images/rasofficial_12256.jpg - Подпись: there is a man in a blue suit holding a microphone
Дистанция: 441.98614501953125 - Изображение: /Applications/Study/Diploma/images/rasofficial_12154.jpg - Подпись: there is a picture of a city with a clock tower
